In [1]:
# Import Dependencies
from path import Path
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler,OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from collections import defaultdict
import pickle, os
import tensorflow as tf


# Preliminary data preprocessing
Original Dataset was provided by a Kaggle Project.  For the intial analysis of the dataset the information was imported to a local directory and then imported read into this jupyter notebook.  Later we will have the connection to the database here. The dataset already came as a csv file so no further processing was necessary.  

Merging of the weather data was accomplished by using the World Weather Online API.  The download of the required information was achieved in a seperate jupyter notebook.  Data was formated as a simple csv file and was imported below.

In [2]:
# Import testing dataset
flights_2019 = pd.read_csv('Database/Data/jan_19_clean.csv')
flights_2020 = pd.read_csv('Database/Data/jan_20_clean.csv')

print('Total Flights for 2019: ', len(flights_2019)
      , '  Total Flights for 2020: ', len(flights_2020))

flights_2019.head()

Total Flights for 2019:  565963   Total Flights for 2020:  599268


,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,1,2,20363,9E,N8688C,3280,11953,1195302,GNV,10397,1039707,ATL,601.0,0.0,0600-0659,722.0,0.0,0,0,300
1,1,2,20363,9E,N348PQ,3281,13487,1348702,MSP,11193,1119302,CVG,1359.0,0.0,1400-1459,1633.0,0.0,0,0,596
2,1,2,20363,9E,N8896A,3282,11433,1143302,DTW,11193,1119302,CVG,1215.0,0.0,1200-1259,1329.0,0.0,0,0,229
3,1,2,20363,9E,N8886A,3283,15249,1524906,TLH,10397,1039707,ATL,1521.0,0.0,1500-1559,1625.0,0.0,0,0,223
4,1,2,20363,9E,N8974C,3284,10397,1039707,ATL,11778,1177801,FSM,1847.0,0.0,1900-1959,1940.0,0.0,0,0,579


In [3]:
flights_2019['Year'] = 2019
flights_2020['Year'] = 2020

In [4]:
len(flights_2020.ORIGIN.unique())

351

In [5]:
# Import Weather Data
weather_2018 = pd.read_csv('Database/weather/weather_data/weather_2019_df.csv')
weather_2019 = pd.read_csv('Database/weather/weather_data/weather_2019_df.csv')
weather_2020 = pd.read_csv('Database/weather/weather_data/weather_2020_df.csv')

# Combine the weather dataframes
weather_df = pd.concat([weather_2018, weather_2019, weather_2020])

weather_df.head()

,location,date_time,precipMM,visibility,cloudcover,windspeedKmph,humidity
0,"islip,ny",2019-01-01 00:00:00,8.5,5,100,10,91
1,"islip,ny",2019-01-01 01:00:00,4.3,5,100,12,93
2,"islip,ny",2019-01-01 02:00:00,4.3,6,100,15,94
3,"islip,ny",2019-01-01 03:00:00,5.8,6,100,18,96
4,"islip,ny",2019-01-01 04:00:00,2.9,7,98,20,96


In [6]:
len(weather_df.location.unique())

318

In [7]:
len(weather_df)

709776

In [8]:
# Import airport city information
locations_lookup_df = pd.read_csv("Flight_data_files/post_hardcode_cities.csv")
locations_lookup_df.drop(columns = "Unnamed: 0", axis = 1, inplace = True)
locations_lookup_df.rename(columns = {"City": "location"}, inplace = True )
locations_lookup_df.head()

,Airport Code,location
0,ABR,"aberdeen,sd"
1,ABI,"abilene,tx"
2,ADK,"adakisland,ak"
3,CAK,"akron,oh"
4,ALB,"albany,ny"


In [9]:
weather_merged = pd.merge(weather_df, locations_lookup_df, on = "location", how = "left")


In [10]:
weather_merged[weather_merged['Airport Code'].isnull()].location.unique()

array([], dtype=object)

In [11]:
weather_merged.date_time = weather_merged.date_time.astype('datetime64[ns]')
weather_merged['Year'] = weather_merged.date_time.dt.year
weather_merged['Month'] = weather_merged.date_time.dt.month
weather_merged['Day'] = weather_merged.date_time.dt.day
weather_merged['Hour'] = weather_merged.date_time.dt.hour

In [12]:
weather_merged.head()

,location,date_time,precipMM,visibility,cloudcover,windspeedKmph,humidity,Airport Code,Year,Month,Day,Hour
0,"islip,ny",2019-01-01 00:00:00,8.5,5,100,10,91,ISP,2019,1,1,0
1,"islip,ny",2019-01-01 01:00:00,4.3,5,100,12,93,ISP,2019,1,1,1
2,"islip,ny",2019-01-01 02:00:00,4.3,6,100,15,94,ISP,2019,1,1,2
3,"islip,ny",2019-01-01 03:00:00,5.8,6,100,18,96,ISP,2019,1,1,3
4,"islip,ny",2019-01-01 04:00:00,2.9,7,98,20,96,ISP,2019,1,1,4


## Preliminary Feature Engineering
Beyond a visual inspection of the head of the data the following steps were performed below:

1) Determined if there are any rows that need to be dropped because they are missing data.

2) Determine which columns of data will need to be included in the training set.

3) Perform the inital encoding of the columns that contained objects as numerical values so it would be easier for the machine learning model to process it.

In [13]:
na_count = flights_2019.DAY_OF_MONTH.count() - flights_2019.dropna(axis=0).DAY_OF_MONTH.count()
print('Total rows with missing information from 2019 data was:  ', na_count )

Total rows with missing information from 2019 data was:   0


In [14]:
na_count = flights_2020.DAY_OF_MONTH.count() - flights_2020.dropna(axis=0).DAY_OF_MONTH.count()
print('Total rows with missing information from 2019 data was:  ', na_count )

Total rows with missing information from 2019 data was:   0


### Step one complete.  The datasets are already cleaned with no missing information in any of the rows.

In [15]:
# Combine the two data sets to give more data for the model to work with
flights = pd.concat([flights_2019, flights_2020])
flights['Month'] = 1

In [16]:
# Preporation to Merge in Weather Data, add hour column
flights['DEP_Hour'] = (flights['DEP_TIME']/100).astype("int")
flights['DEP_Hour'] = flights['DEP_Hour'].replace(24, 0) # replace 24 hours with 0 to match weather data


In [17]:
#  Look at the available columns
flights.columns

Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER',
       'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID',
       'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN', 'DEST_AIRPORT_ID',
       'DEST_AIRPORT_SEQ_ID', 'DEST', 'DEP_TIME', 'DEP_DEL15', 'DEP_TIME_BLK',
       'ARR_TIME', 'ARR_DEL15', 'CANCELLED', 'DIVERTED', 'DISTANCE', 'Year',
       'Month', 'DEP_Hour'],
      dtype='object')

In [18]:
len(flights)

1165231

In [19]:
flights_merged = pd.merge(flights, weather_merged
                          ,  how='left'
                          , left_on=['ORIGIN', 'Year', 'Month', 'DAY_OF_MONTH' , 'DEP_Hour']
                          , right_on = ['Airport Code', 'Year', 'Month', 'Day', 'Hour'])

In [20]:
len(flights_merged)

1720680

In [21]:
filteredList = ['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_CARRIER', 'ORIGIN', 'DEST', 'DEP_DEL15', 'DISTANCE'
                ,'DEP_TIME_BLK', 'TAIL_NUM']

flights_merged = flights_merged.drop_duplicates(subset = filteredList)

In [22]:
len(flights_merged)

1165231

In [23]:
flights_merged.head()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,DEST_AIRPORT_ID,...,location,date_time,precipMM,visibility,cloudcover,windspeedKmph,humidity,Airport Code,Day,Hour
0,1,2,20363,9E,N8688C,3280,11953,1195302,GNV,10397,...,"gainesville,fl",2019-01-01 06:00:00,0.0,10.0,31.0,4.0,97.0,GNV,1.0,6.0
2,1,2,20363,9E,N348PQ,3281,13487,1348702,MSP,11193,...,"minneapolis,mn",2019-01-01 13:00:00,0.0,10.0,56.0,10.0,68.0,MSP,1.0,13.0
4,1,2,20363,9E,N8896A,3282,11433,1143302,DTW,11193,...,"detroit,mi",2019-01-01 12:00:00,0.0,7.0,100.0,21.0,77.0,DTW,1.0,12.0
6,1,2,20363,9E,N8886A,3283,15249,1524906,TLH,10397,...,"tallahassee,fl",2019-01-01 15:00:00,0.0,10.0,52.0,10.0,80.0,TLH,1.0,15.0
8,1,2,20363,9E,N8974C,3284,10397,1039707,ATL,11778,...,"atlanta,ga",2019-01-01 18:00:00,2.4,7.0,88.0,10.0,74.0,ATL,1.0,18.0


In [24]:
missing_Airports = flights_merged[flights_merged['Airport Code'].isnull()].ORIGIN.unique()

In [25]:
missing_Airports

array(['EWN', 'DRT', 'GRK', 'SWO', 'AZA', 'PGD', 'SFB', 'PBG', 'USA',
       'IAG', 'STX', 'BQN', 'STT', 'HOB', 'ECP', 'ACY', 'DAL', 'FCA',
       'ABY', 'MMH', 'LCK', 'GUM', 'SPN', 'PVU', 'RFD', 'PPG', 'OGD',
       'LYH', 'XWA', 'PAE', 'BFM'], dtype=object)

In [26]:
# Removing airports that do not have weather data
flights_cleaned_merged = flights_merged.dropna()
len(flights_merged) - len(flights_cleaned_merged)

21645

In [27]:
flights_cleaned_merged.columns

Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER',
       'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID',
       'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN', 'DEST_AIRPORT_ID',
       'DEST_AIRPORT_SEQ_ID', 'DEST', 'DEP_TIME', 'DEP_DEL15', 'DEP_TIME_BLK',
       'ARR_TIME', 'ARR_DEL15', 'CANCELLED', 'DIVERTED', 'DISTANCE', 'Year',
       'Month', 'DEP_Hour', 'location', 'date_time', 'precipMM', 'visibility',
       'cloudcover', 'windspeedKmph', 'humidity', 'Airport Code', 'Day',
       'Hour'],
      dtype='object')

In [28]:
# get top 50 most frequent ORIGINS
n = 50
top_50_ORIGIN = flights_cleaned_merged.ORIGIN.value_counts()[:n].index.tolist()

In [47]:
# Output the top 50 for use for the dashboard
top_50 = pd.DataFrame((flights_cleaned_merged.groupby('ORIGIN').ORIGIN.count().sort_values(ascending=False)[:50]))
top_50.index.name = "Airport"
top_50.columns = ['flight_count']
top_50.to_csv("top_50_ORIGIN_Airports.csv")


In [48]:
top_50

,flight_count
Airport,
ATL,62807
ORD,48102
DFW,46523
CLT,38641
DEN,38373
LAX,35307
PHX,29731
IAH,29140
LAS,27132


In [287]:
flights_cleaned_filtered = flights_cleaned_merged[flights_cleaned_merged.ORIGIN.isin(top_50_ORIGIN)]

In [288]:
len(flights_cleaned_filtered)

910877

### Step 2:
#### Feature Selection:

To determine which features to include it was important to determine how we want to use the predictive model.
In this case we will be taking a set of inputs from the user and displaying which time slot is most likely to not
have a flight delay. So we will only include information the model that we can get from the user or provide look up tables for.

The filterdList below included all the data that we either we will be able to get from the user, will be a look up
or is the primary feature we want to predict. 

In this case it is whether there will be a departure delay or not the column name DEP_DEL15 will the be feature we want to predict.

In [746]:
# Create the list that we are going to keep as features
filteredList = ['OP_CARRIER', 'ORIGIN', 'DEST', 'DEP_DEL15'
                ,'DEP_Hour', 'windspeedKmph', 'precipMM']

# Keep the list of columns that were removed...possibly for later use.
columnsRemoved = ['DAY_OF_WEEK', 'DAY_OF_MONTH', 'visibility', 'OP_CARRIER_AIRLINE_ID', 'DEST_AIRPORT_SEQ_ID', 'ORIGIN_AIRPORT_SEQ_ID' , 'ORIGIN_AIRPORT_ID', 'DISTANCE', 'TAIL_NUM'
                  , 'OP_CARRIER_FL_NUM', 'DEST_AIRPORT_ID', 'ARR_TIME', 'ARR_DEL15', 'CANCELLED', 'DIVERTED', 'DEP_TIME', 'DEP_TIME_BLK']

# Filter the data to only include the columns we want
machine_model_df = flights_cleaned_filtered.filter(filteredList)


In [747]:
# Set the feature we want to predict as a integer since on import it was made a float
machine_model_df.DEP_DEL15 = machine_model_df.DEP_DEL15.astype('int')


In [748]:
machine_model_df.head()

,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEST,DEP_DEL15,DEP_Hour,windspeedKmph,precipMM
2,2,9E,MSP,CVG,0,13,10.0,0.0
4,2,9E,DTW,CVG,0,12,21.0,0.0
8,2,9E,ATL,FSM,0,18,10.0,2.4
14,2,9E,LGA,CVG,0,15,19.0,0.1
18,2,9E,ATL,BMI,0,12,12.0,0.1


In [749]:
machine_model_df.dtypes

DAY_OF_WEEK        int64
OP_CARRIER        object
ORIGIN            object
DEST              object
DEP_DEL15          int32
DEP_Hour           int32
windspeedKmph    float64
precipMM         float64
dtype: object

In [750]:
# We are evaluating the impact of including Departure Time and a floating number in the dataset.
# if it is there then make it an integer
if "DEP_TIME" in machine_model_df.columns:
    machine_model_df.DEP_TIME = machine_model_df.DEP_TIME.astype('int')

In [751]:
# Examine features and determine how the various features will need to be encoded
machine_model_df.dtypes

DAY_OF_WEEK        int64
OP_CARRIER        object
ORIGIN            object
DEST              object
DEP_DEL15          int32
DEP_Hour           int32
windspeedKmph    float64
precipMM         float64
dtype: object

In [752]:
# Generate our categorical variable list
flights_cat = machine_model_df.dtypes[machine_model_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
machine_model_df[flights_cat].nunique()

OP_CARRIER     17
ORIGIN         50
DEST          334
dtype: int64

In [753]:
# Evaluate how many records are available in the dataset
print('Total number of records:  ', machine_model_df.ORIGIN.count())
print('Total number of columns:  ', len(machine_model_df.columns))

Total number of records:   910877
Total number of columns:   8


###  Step 2 - Encoding:
#### Encoding Method:
A couple of things to note, the complete dataset has over a million records and there are currently 9 columns.  Given the number of columns that have unique string data, to attempt to apply get_dummies would dramatically increase the datatable size.  This was attempted on the Carrier and Origin the result was a error code stating that there was inadequate resources.  

We will reserve the get_dummy encoding method for the OP_Carrier and Departure Time Blocks only and will use LabelEncoder for Origin, Destination, and Tail Number.  It is necessary to not limit ourselves to only the top ten of any of these options because these will be unique entries by the user later when the model is being implemented.

In [754]:
# Perform get_dummies method on the OP_Carrier and Departure Time Blocks columns
dummy_columns = ['OP_CARRIER']
prefix = ['Carrier']
dummy_df = pd.get_dummies(machine_model_df[dummy_columns], prefix)

In [755]:
# Tranform each column with LabelEncoder (https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn)
lableEncoder_columns = ['ORIGIN', 'DEST']  #, 'TAIL_NUM'
d = defaultdict(LabelEncoder)

df = machine_model_df[lableEncoder_columns]

# Encoding the variable
labelEncoded_df = df.apply(lambda x: d[x.name].fit_transform(x))

# Retaining this code here for later reference
## Inverse the encoded
# fit.apply(lambda x: d[x.name].inverse_transform(x))

## Using the dictionary to label future data
# df.apply(lambda x: d[x.name].transform(x))

In [756]:
# Concatenate the encoded columns
machine_model_df_encoded = machine_model_df.drop(flights_cat, axis=1)
machine_model_df_encoded = pd.concat([machine_model_df_encoded, dummy_df, labelEncoded_df], axis = 1)
machine_model_df_encoded.head()

,DAY_OF_WEEK,DEP_DEL15,DEP_Hour,windspeedKmph,precipMM,Carrier_9E,Carrier_AA,Carrier_AS,Carrier_B6,Carrier_DL,...,Carrier_MQ,Carrier_NK,Carrier_OH,Carrier_OO,Carrier_UA,Carrier_WN,Carrier_YV,Carrier_YX,ORIGIN,DEST
2,2,0,13,10.0,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,29,77
4,2,0,12,21.0,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,13,77
8,2,0,18,10.0,2.4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,116
14,2,0,15,19.0,0.1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,24,77
18,2,0,12,12.0,0.1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,37


In [775]:
machine_model_df_encoded.groupby(['DEP_DEL15']).size()

DEP_DEL15
0    768487
1    142390
dtype: int64

In [757]:
# Create Feature and Target Variables
y = machine_model_df_encoded["DEP_DEL15"]  # Target
X = machine_model_df_encoded.drop(columns="DEP_DEL15") # Features

##  Splitting Data:
Using the method imported from the package sklearn, the dataset is split into a training set and a test set.
The seperation is achived randomly.  
By adding the condition stratify = y, we are insuring that the test and training sets contain the proportion 
of values as provided in the target set.  By setting random_state to a specific value we can get repeatedly the same split of the data so it is possible to make sure that at least the testing and training sets are consistent.

In [758]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=34, 
                                                    stratify=y)
X_train.shape

(683157, 23)

In [759]:
# Preprocess numerical data

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Model Evaluation

Multiple models were evaluated to determine how well they are able to predict the end result.  It is important to note that there are far fewer delayed flights than flights that are on time.  Because that is true, when evaluating how accurate the model is at predicting we will need to evaluate it using precision, recall, and F1-score.

### LogisticRegression Model

In [760]:
# Create the Logistic Regression Model
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=2000,
                                random_state=10)

In [761]:
# Fit the model
classifier.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=2000, random_state=10)

In [762]:
# Make predictions 
y_pred = classifier.predict(X_test_scaled)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [763]:
metrics.confusion_matrix(y_test, y_pred)

array([[192059,     63],
       [ 35561,     37]], dtype=int64)

In [764]:
# Because dataset is imbalanced we cant use accuracy metric
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      1.00      0.92    192122
           1       0.37      0.00      0.00     35598

    accuracy                           0.84    227720
   macro avg       0.61      0.50      0.46    227720
weighted avg       0.77      0.84      0.77    227720



# Decision Tree Analysis

In [765]:
clf_dt = DecisionTreeClassifier(random_state = 0)
model_dt = clf_dt.fit(X_train_scaled, y_train)

In [766]:
y_pred = model_dt.predict(X_test_scaled)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88    192122
           1       0.35      0.35      0.35     35598

    accuracy                           0.80    227720
   macro avg       0.62      0.62      0.62    227720
weighted avg       0.80      0.80      0.80    227720



In [767]:
metrics.confusion_matrix(y_test, y_pred)

array([[169123,  22999],
       [ 23114,  12484]], dtype=int64)

In [768]:
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,0,0
1,1,0
2,0,0
3,1,0
4,0,0
5,0,0
6,1,0
7,0,0
8,1,0
9,0,0


# Random Forest Classifier
This one is too slow and is demonstrated to be a poor predictor

In [772]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(max_depth=20)
model_rf = clf_rf.fit(X_train_scaled, y_train)

In [773]:
y_pred = model_rf.predict(X_test_scaled)
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92    192122
           1       0.80      0.07      0.13     35598

    accuracy                           0.85    227720
   macro avg       0.82      0.53      0.52    227720
weighted avg       0.84      0.85      0.79    227720



In [774]:
metrics.confusion_matrix(y_test, y_pred)

array([[191500,    622],
       [ 33166,   2432]], dtype=int64)

In [779]:
class_weight = dict({1:1, 2:6})
rdf = RandomForestClassifier(bootstrap=True,
            class_weight=class_weight, 
            criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=300,
            oob_score=False,
            random_state=0,
            verbose=0, warm_start=False)
model_rdf = rdf.fit(X_train_scaled, y_train)

ValueError: Class label 2 not present.

### NeuralNetwork

In [50]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  1.5 * len(X_train_scaled[0])
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
27311/27311 [==============================] - 18s 668us/step - loss: 0.4174 - accuracy: 0.8453
Epoch 2/100
27311/27311 [==============================] - 18s 662us/step - loss: 0.4140 - accuracy: 0.8457
Epoch 3/100
27311/27311 [==============================] - 18s 676us/step - loss: 0.4127 - accuracy: 0.8457
Epoch 4/100
27311/27311 [==============================] - 19s 681us/step - loss: 0.4119 - accuracy: 0.8457
Epoch 5/100
27311/27311 [==============================] - 18s 652us/step - loss: 0.4110 - accuracy: 0.8457
Epoch 6/100
27311/27311 [==============================] - 19s 701us/step - loss: 0.4102 - accuracy: 0.8457
Epoch 7/100
27311/27311 [==============================] - 21s 757us/step - loss: 0.4096 - accuracy: 0.8457
Epoch 8/100
27311/27311 [==============================] - 18s 671us/step - loss: 0.4091 - accuracy: 0.8457
Epoch 9/100
27311/27311 [==============================] - 18s 661us/step - loss: 0.4088 - accuracy: 0.8457
Epoch 10/100
27311/27311 [==

In [67]:
# Get Predictions
y_pred = tf.round(nn.predict(X_test_scaled))


In [69]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92    246370
           1       0.52      0.01      0.01     44938

    accuracy                           0.85    291308
   macro avg       0.68      0.50      0.47    291308
weighted avg       0.80      0.85      0.78    291308



In [70]:
metrics.confusion_matrix(y_test, y_pred)

array([[246083,    287],
       [ 44625,    313]], dtype=int64)

### Model Final Selection
After evaluating each model using the entire combined dataset, it was found that Decision Tree was the best at predicting delays.  That will be the model we will use going forward.